In [1]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import lite

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
base_dir = '/home/vasu/Downloads/rockpapersc'

train_dir = os.path.join(base_dir, 'Training')
validation_dir = os.path.join(base_dir, 'Testing')

In [3]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    
    tf.keras.layers.Dense(3, activation='softmax')  
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])

In [5]:
train_datagen = ImageDataGenerator( rescale = 1.0/255.,
                                   horizontal_flip=True,
                                   height_shift_range=0.3,
                                   width_shift_range=0.3,
                                   rotation_range=0.3,
                                   shear_range=0.2,
                                   zoom_range=0.3)
test_datagen  = ImageDataGenerator( rescale = 1.0/255.)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    target_size=(150, 150),
                                                    class_mode='categorical')     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         target_size = (150, 150),
                                                         class_mode='categorical')

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [7]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=1,
                              validation_steps=50,
                              verbose=2)

100/100 - 42s - loss: 1.0243 - acc: 0.4620 - val_loss: 0.7665 - val_acc: 0.8465


In [26]:
#saving the model
tf.keras.models.save_model(model,"/home/vasu/Downloads/rockpapersc/model.h5")
kerasmodel=tf.keras.models.load_model("/home/vasu/Downloads/rockpapersc/model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [39]:
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(kerasmodel)
#tflite_model = converter.convert()

AttributeError: type object 'TFLiteConverter' has no attribute 'from_keras_model'

In [8]:
tf.saved_model.save(model,base_dir)
tflite_converter = tf.lite.TFLiteConverter.from_saved_model(base_dir)
tflite_model = tflite_converter.convert()

INFO:tensorflow:Assets written to: /home/vasu/Downloads/rockpapersc/assets
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/vasu/Downloads/rockpapersc/variables/variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'__saved_model_init_op', 'serving_default'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: conv2d_input
INFO:tensorflow: tensor name: serving_default_conv2d_input:0, shape: (-1, 150, 150, 3), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: dense_1
INFO:tensorflow: tensor name: Statef

ConverterError: TOCO failed. See console for info.
/home/vasu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vasu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vasu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vasu/.local/lib/python3.7/site-packages/